### 62101046 雨宮佳音 自然言語処理レポート課題

# 1. 準備
必要なライブラリをインストール・インポートする

In [5]:
!pip install nltk
!pip install matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# 2. 品詞のタグ付け
ある文章のそれぞれのトークンに品詞を割り当てるタスクを考える。ここでは5種類のタガーをそれぞれ評価することで、優れたタグ付け方法を模索する。

### 2-1. 準備

In [6]:
import nltk
from nltk.corpus import brown
from nltk.metrics import precision, recall, f_measure
nltk.download('brown')

# 文章と正解データの準備
sentences = brown.sents(categories="news")
tagged_sentences = brown.tagged_sents(categories="news")

# 品詞を比較して評価する関数
def evaluate_tagger(tagger, sentences, tagged_sentences):
    predicted_tags = [tagger.tag(sent) for sent in sentences]
    reference_tags = [[tag for (word, tag) in tagged_sent] for tagged_sent in tagged_sentences]
    print("予測データの一部: ", predicted_tags[0])
    print("正解データの一部: ", reference_tags[0])

    # 品詞の一致数を計算
    correct_tags = sum(sum(1 for (predicted_word, predicted_tag), reference_tag in zip(predicted_sent, reference_sent) if predicted_tag == reference_tag)
                  for predicted_sent, reference_sent in zip(predicted_tags, reference_tags))


    # 評価を計算
    total_tags = sum(len(tags) for tags in reference_tags)
    predicted_tags_flat = [tag for sent in predicted_tags for _, tag in sent]
    reference_tags_flat = [tag for sent in reference_tags for tag in sent]

    accuracy = correct_tags / total_tags
    precision_score = precision(set(predicted_tags_flat), set(reference_tags_flat))
    recall_score = recall(set(predicted_tags_flat), set(reference_tags_flat))
    f_measure_score = f_measure(set(predicted_tags_flat), set(reference_tags_flat))

    print("Accuracy: {:.4f}".format(accuracy))
    print("Precision: {:.4f}".format(precision_score))
    print("Recall: {:.4f}".format(recall_score))
    print("F-measure: {:.4f}".format(f_measure_score))

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!


### 2-2. デフォルトタガー
デフォルトタガーとは、すべてのトークンに同じタグをつけるタグ付け方式である。
タグ付きコーパスから最頻出の品詞を調べ、それをすべての単語に適用する。

In [7]:
# default_taggerを用意
tags = [tag for (word, tag) in brown.tagged_words(categories="news")]
most_frequent_tag = nltk.FreqDist(tags).max()
default_tagger = nltk.DefaultTagger(most_frequent_tag)

# 評価を実行
evaluate_tagger(default_tagger, sentences, tagged_sentences)

予測データの一部:  [('The', 'NN'), ('Fulton', 'NN'), ('County', 'NN'), ('Grand', 'NN'), ('Jury', 'NN'), ('said', 'NN'), ('Friday', 'NN'), ('an', 'NN'), ('investigation', 'NN'), ('of', 'NN'), ("Atlanta's", 'NN'), ('recent', 'NN'), ('primary', 'NN'), ('election', 'NN'), ('produced', 'NN'), ('``', 'NN'), ('no', 'NN'), ('evidence', 'NN'), ("''", 'NN'), ('that', 'NN'), ('any', 'NN'), ('irregularities', 'NN'), ('took', 'NN'), ('place', 'NN'), ('.', 'NN')]
正解データの一部:  ['AT', 'NP-TL', 'NN-TL', 'JJ-TL', 'NN-TL', 'VBD', 'NR', 'AT', 'NN', 'IN', 'NP$', 'JJ', 'NN', 'NN', 'VBD', '``', 'AT', 'NN', "''", 'CS', 'DTI', 'NNS', 'VBD', 'NN', '.']
Accuracy: 0.1309
Precision: 0.0046
Recall: 1.0000
F-measure: 0.0091


当然、評価は低く、精度は13.09%であった。予測データの一部を見ると、すべての単語に「NN」が適用されていることがわかる。つまり、文章全体の品詞のうち「NN」が最頻出で13.09%もを占めていることがわかる。正解値が「NN」であるものをすべて再現できていることから、再現率は100%となっている。

### 2-3. 正規表現タガー
正規表現タガーとは、パターンマッチに基づくタグ付け方法である。正規表現で単語の語尾を指定し、それに当てはまれば該当の品詞を振る。

In [8]:
print("参考書の記述に沿ったパターン")

# regexp_tagger_1を用意
patterns_1 = [
    (r".*ing$", "VBG"),
    (r".*ed$", "VBD"),
    (r".*es$", "VBZ"),
    (r".*ould$", "MD"),
    (r".*\'s$", "NN$"),
    (r".*s$", "NNS"),
    (r"^-?[0-9]+(.[0-9]+)?$", "CD"),
    (r".*", "NN")
]
regexp_tagger_1 = nltk.RegexpTagger(patterns_1)

# 評価を実行
evaluate_tagger(regexp_tagger_1, sentences, tagged_sentences)

print("上記に自分で追加したパターン")

# regexp_tagger_2を用意
patterns_2 = [
    (r"^([A-Z][a-z]+\.?)+$", "NNP"),  # 固有名詞
    (r"^-?[0-9]+(.[0-9]+)?$", "CD"),  # 数字
    (r".*s$", "NNS"),  # 複数形名詞
    (r".*ing$", "VBG"),  # 動詞の進行形
    (r".*ed$", "VBD"),  # 動詞の過去形
    (r".*es$", "VBZ"),  # 動詞の三人称単数現在形
    (r".*ould$", "MD"),  # 助動詞
    (r".*\'s$", "NN$"),  # 名詞所有格
    (r".*ment$", "NN"),  # 名詞の語尾「ment」
    (r".*ful$", "JJ"),  # 形容詞の語尾「ful」
    (r".*ly$", "RB"),  # 副詞の語尾「ly」
    (r".*", "NN")  # デフォルトの名詞
]
regexp_tagger_2 = nltk.RegexpTagger(patterns_2)

# 評価を実行
evaluate_tagger(regexp_tagger_2, sentences, tagged_sentences)

参考書の記述に沿ったパターン
予測データの一部:  [('The', 'NN'), ('Fulton', 'NN'), ('County', 'NN'), ('Grand', 'NN'), ('Jury', 'NN'), ('said', 'NN'), ('Friday', 'NN'), ('an', 'NN'), ('investigation', 'NN'), ('of', 'NN'), ("Atlanta's", 'NN$'), ('recent', 'NN'), ('primary', 'NN'), ('election', 'NN'), ('produced', 'VBD'), ('``', 'NN'), ('no', 'NN'), ('evidence', 'NN'), ("''", 'NN'), ('that', 'NN'), ('any', 'NN'), ('irregularities', 'VBZ'), ('took', 'NN'), ('place', 'NN'), ('.', 'NN')]
正解データの一部:  ['AT', 'NP-TL', 'NN-TL', 'JJ-TL', 'NN-TL', 'VBD', 'NR', 'AT', 'NN', 'IN', 'NP$', 'JJ', 'NN', 'NN', 'VBD', '``', 'AT', 'NN', "''", 'CS', 'DTI', 'NNS', 'VBD', 'NN', '.']
Accuracy: 0.2033
Precision: 0.0367
Recall: 1.0000
F-measure: 0.0708
上記に自分で追加したパターン
予測データの一部:  [('The', 'NNP'), ('Fulton', 'NNP'), ('County', 'NNP'), ('Grand', 'NNP'), ('Jury', 'NNP'), ('said', 'NN'), ('Friday', 'NNP'), ('an', 'NN'), ('investigation', 'NN'), ('of', 'NN'), ("Atlanta's", 'NNS'), ('recent', 'NN'), ('primary', 'NN'), ('election', 'NN'), ('prod

デフォルトタガーが13.09%だったのに対し、正規表現タガーでは精度が上がった。参考書に書いてあるパターンをそのまま記述した場合20.33%、パターンを追加したり順序を変更したりした場合21.58%になった。自分で追加したパターンには、固有名詞、名詞「ment」、形容詞「ful」、副詞「ly」を追加した。順序については、固有名詞や数字など特殊なパターンを先に配置し、一般的なパターンを後に置くことで、特殊なパターンがより一般的なパターンに優先されることを防いだ。ただし、再現率が大きく下がっていることに注意する。条件を細かくしたことで、本来より一般的な品詞としてタグづけされるべきであったトークンが、途中で誤って拾われたと考える。F値が僅かに下がっているため、必ずしも良いタガーになったとは言えない。

### 2-4. ユニグラムタガー
ユニグラムタガーとは、各トークンについて、そのトークンに対して最も多く付けられている品詞をあてるタグ付け方法である。これ以降では「tagged_sentences」を訓練集合とテスト集合に分けて、テスト集合において評価をするため、上2つのタガーとは評価基準が異なることに注意する。

In [9]:
# unigram_taggerを用意
size = int(len(tagged_sentences) * 0.9)
train_sentences = tagged_sentences[:size]
test_tagged_sentences = tagged_sentences[size:]
test_sentences = sentences[size:]
unigram_tagger = nltk.UnigramTagger(train_sentences)

# 評価を実行
evaluate_tagger(unigram_tagger, test_sentences, test_tagged_sentences)

予測データの一部:  [('But', 'CC'), ('in', 'IN'), ('all', 'ABN'), ('its', 'PP$'), ('175', None), ('years', 'NNS'), (',', ','), ('not', '*'), ('a', 'AT'), ('single', 'AP'), ('Negro', 'NP'), ('student', 'NN'), ('has', 'HVZ'), ('entered', 'VBD'), ('its', 'PP$'), ('classrooms', None), ('.', '.')]
正解データの一部:  ['CC', 'IN', 'ABN', 'PP$', 'CD', 'NNS', ',', '*', 'AT', 'AP', 'NP', 'NN', 'HVZ', 'VBN', 'PP$', 'NNS', '.']
Accuracy: 0.8121
Precision: 0.7479
Recall: 0.9468
F-measure: 0.8357


81.21%という非常に高い精度になった。つまり、81.21%のトークンは、そのトークンの最頻出品詞として使われていることがわかる。ユニグラムに基づいたダグ付けの訓練では、現在処理しているトークンだけを、その他の文脈からは分離して考える。そのため、「wind」のように名詞にも動詞にもなりうる単語は、「the wind」であっても「to wind」であっても、同じタグをつけてしまう。そこで、前の単語の品詞の情報をもとに文脈を考慮できるようバイグラムタガーを考える。

### 2-5. バイグラムタガー
バイグラムタガーとは、現在処理しているトークンとその1つ前のトークンとの組み合わせに対して、最も多く付けられている品詞をあてるタグ付け方法である。文脈を考慮できるため、「wind」などの区別もできるようになり、ユニグラムタガーより精度が高くなる。

In [10]:
# bigram_taggerを用意
bigram_tagger = nltk.BigramTagger(train_sentences)

# 評価を実行
evaluate_tagger(bigram_tagger, test_sentences, test_tagged_sentences)

予測データの一部:  [('But', 'CC'), ('in', 'IN'), ('all', 'ABN'), ('its', 'PP$'), ('175', None), ('years', None), (',', None), ('not', None), ('a', None), ('single', None), ('Negro', None), ('student', None), ('has', None), ('entered', None), ('its', None), ('classrooms', None), ('.', None)]
正解データの一部:  ['CC', 'IN', 'ABN', 'PP$', 'CD', 'NNS', ',', '*', 'AT', 'AP', 'NP', 'NN', 'HVZ', 'VBN', 'PP$', 'NNS', '.']
Accuracy: 0.1021
Precision: 0.5714
Recall: 0.9577
F-measure: 0.7158


精度が10.21%という非常に低い結果になってしまった。原因は、予測データの一部を見ればわかる通り、「175」というトークン以降、この1文すべてのトークンの品詞が「None」となっていることである。バイグラムタガーは未知の単語に出会うと、タグ付けを行うことができない。そして、それに続く単語について、それ自身が未知でない場合でも直前が「None」であることから予測できていない。なぜなら、直前に「None」のあるケースが訓練されていないからである。この連鎖が1文の中で最後まで続くため、結果として精度が低くなった。

### 2-6. 組み合わせタガー
より精度の高いアルゴリズムを使えるときは使い、必要な時にはより適用範囲の広いアルゴリズムにフォールバックする方法である。ここでは参考書にあった「バイグラムタガー＆ユニグラムタガー＆デフォルトタガー」に加え、「トライグラムタガー＆バイグラムタガー＆ユニグラムタガー&正規表現タガー」を実装する。前者の例では、初めにバイグラムタガーを使ってトークンのタグ付けをし、それではタグが発見できなかった場合はユニグラムタガーを使い、それでも発見できなかった場合はデフォルトタガーを使うと言う仕組みである。

In [11]:
print("参考書の記述に沿った組み合わせ")

# combination_taggerを用意
combination_tagger_1 = nltk.UnigramTagger(train_sentences, backoff=default_tagger)
combination_tagger_2 = nltk.BigramTagger(train_sentences, backoff=combination_tagger_1)

# 評価を実行
evaluate_tagger(combination_tagger_2, test_sentences, test_tagged_sentences)

print("自分で考えた組み合わせ")
# combination_taggerを用意
combination_tagger_1 = nltk.UnigramTagger(train_sentences, backoff=regexp_tagger_2)
combination_tagger_2 = nltk.BigramTagger(train_sentences, backoff=combination_tagger_1)
combination_tagger_3 = nltk.TrigramTagger(train_sentences, backoff=combination_tagger_2)

# 評価を実行
evaluate_tagger(combination_tagger_2, test_sentences, test_tagged_sentences)

参考書の記述に沿った組み合わせ
予測データの一部:  [('But', 'CC'), ('in', 'IN'), ('all', 'ABN'), ('its', 'PP$'), ('175', 'NN'), ('years', 'NNS'), (',', ','), ('not', '*'), ('a', 'AT'), ('single', 'AP'), ('Negro', 'NP'), ('student', 'NN'), ('has', 'HVZ'), ('entered', 'VBD'), ('its', 'PP$'), ('classrooms', 'NN'), ('.', '.')]
正解データの一部:  ['CC', 'IN', 'ABN', 'PP$', 'CD', 'NNS', ',', '*', 'AT', 'AP', 'NP', 'NN', 'HVZ', 'VBN', 'PP$', 'NNS', '.']
Accuracy: 0.8452
Precision: 0.7983
Recall: 0.9500
F-measure: 0.8676
自分で考えた組み合わせ
予測データの一部:  [('But', 'CC'), ('in', 'IN'), ('all', 'ABN'), ('its', 'PP$'), ('175', 'CD'), ('years', 'NNS'), (',', ','), ('not', '*'), ('a', 'AT'), ('single', 'AP'), ('Negro', 'NP'), ('student', 'NN'), ('has', 'HVZ'), ('entered', 'VBD'), ('its', 'PP$'), ('classrooms', 'NNS'), ('.', '.')]
正解データの一部:  ['CC', 'IN', 'ABN', 'PP$', 'CD', 'NNS', ',', '*', 'AT', 'AP', 'NP', 'NN', 'HVZ', 'VBN', 'PP$', 'NNS', '.']
Accuracy: 0.8725
Precision: 0.7983
Recall: 0.9406
F-measure: 0.8636


タガーを組み合わせて受け皿を作ることにより、バイグラムのみのときの欠点を解消できた。参考書に沿った組み合わせでは84.52%、自分で考えた組み合わせでは87.25%の精度を出すことができた。後者ではトライグラムを使うことで、2トークンの文脈からは推定できなかった品詞を、3トークンの文脈から正しく判断できるようになった。また、最下位のバックオフとして正規表現タガーを使うことで、例えば予測データの「175」を正しく判断できるようになった。

# 3. 文章のチャンキング
1つの文章を複数のトークンで構成されたテキスト断片に分割し、ラベル付けすることをチャンキングという。ここでは5種類のチャンカをそれぞれ評価することで、優れたタグ付け方法を模索する。
### 3-1. チャンキングの視覚的理解
ここでは、最も単純なNPチャンクを実装し、視覚的に理解を深める。

In [12]:
sentence = [("the", "DT"), ("litte", "JJ"), ("yello", "JJ"), ("dog", "NN"), ("barked", "VBD"), ("at", "IN"), ("the", "DT"), ("cat", "NN")]
grammer = "NP: {<DT>?<JJ>*<NN>}"
cp = nltk.RegexpParser(grammer)
print(cp.parse(sentence))


(S
  (NP the/DT litte/JJ yello/JJ dog/NN)
  barked/VBD
  at/IN
  (NP the/DT cat/NN))


![名詞句チャンキングの構造木](https://drive.google.com/uc?id=1W3uv5ATlu9xV1KcwEihRlE5iA03E0-vt)  

ここでは、1つの正規表現のルールで構成された単純なチャンク文法を使っている。このルールにおいてNPチャンクは、省略可能な限定詞（DT）に続いて任意の数の形容詞（JJ）が出現し、最後に名詞（NN）が来るものと定義されている。  

次に2つのルールで構成されるNPチャンクを実装する。

In [13]:
sentence = [("Rapunzel", "NNP"), ("let", "VBD"), ("down", "RP"), ("her", "PP$"), ("long", "JJ"), ("gorlden", "JJ"), ("hair", "NN")]
grammer = r"""
    NP: {<DT|PP\$>?<JJ>*<NN>}
        {<NNP>+}
"""
cp = nltk.RegexpParser(grammer)
print(cp.parse(sentence))

(S
  (NP Rapunzel/NNP)
  let/VBD
  down/RP
  (NP her/PP$ long/JJ gorlden/JJ hair/NN))


![名詞句チャンキングの構造木](https://drive.google.com/uc?id=1iHCyGjacg99UmWgUYrNOlHKP5XenVUvk)  

1つ目のルールにおいては、NPチャンクは、0あるいは1個の限定詞もしくは所有代名詞、0個以上の形容詞、1つの名詞だと定義づけられている。2つ目のルールにおいては、1つ以上の固有名詞だと定義づけられている。

### 3-2. IOBタグ
IOBタグによって、それぞれのトークンはI(inside)、O(outside)、B(begin)と言う3つの特殊なチャンクダグが付けられる。チャンクの先頭に存在するトークンにはB、それに続くチャンク内にあるトークンにはI、それ以外のチャンク外のトークンにはOというタグが付けられる。BとIのタグには、B-NP、I-NPのように、チャンクタイプが接尾辞として付けられる。

### 3-3. 準備

In [14]:
from nltk.corpus import conll2000
nltk.download('conll2000')

# データの準備
train_sentences = conll2000.chunked_sents("test.txt", chunk_types=["NP"])
test_sentences = conll2000.chunked_sents("test.txt", chunk_types=["NP"])
postags = sorted(set(pos for sent in train_sentences
                    for (word, pos) in sent.leaves()))


[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Unzipping corpora/conll2000.zip.


### 3-4. 最も単純なチャンカ
「nltk.RegexpParser()」の引数に何も指定せずに、全くチャンクを生成しないチャンカを作成した。

In [15]:
cp = nltk.RegexpParser("")

print(cp.evaluate(test_sentences))

<ipython-input-15-78ee0edddd02>:3: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  print(cp.evaluate(test_sentences))


ChunkParse score:
    IOB Accuracy:  43.4%%
    Precision:      0.0%%
    Recall:         0.0%%
    F-Measure:      0.0%%


このチャンカがチャンキングを全く行わないことから、「IOB Accuracy:  43.4%」と言う結果は、43.4%の単語にOのタグが付けられており、それらがNPチャンクではないことを表す。そして、このチャンカはチャンクを全く見つけることができないため、適応率、再現率、F値はいずれも0になる。

### 3-5. 単純な正規表現チャンカ
名詞句タグを特徴的に表す文字で始まるタグを発見する単純な正規表現チャンカを作成した。

In [16]:
grammer = r"NP: {<[CDJNP].*>+}"
cp = nltk.RegexpParser(grammer)

print(cp.evaluate(test_sentences))

<ipython-input-16-2a230905174b>:4: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  print(cp.evaluate(test_sentences))


ChunkParse score:
    IOB Accuracy:  87.7%%
    Precision:     70.6%%
    Recall:        67.8%%
    F-Measure:     69.2%%


「<[CDJNP].*>+」より、「C」「D」「J」「N」「P」のいずれかの品詞タグから始まる単語の、1回以上の繰り返しで構成される名詞句をチャンク化する。非常に簡単な正規表現しか使っていないが、比較的高い精度を出すことがわかる。一方で、適合率と再現率の低さから、一部の名詞句を見逃していたり、誤った部分もチャンクとして予測したりする箇所が多いことがわかる。

### 3-6. ユニグラムタガーを用いた名詞句チャンカ
訓練コーパスを利用してそれぞれの品詞タグに最も適切なチャンクタグを探すため、ユニグラムタガーを用いる。このユニグラムタガーは、それぞれの単語の正しい品詞タグを決定するのではなく、品詞タグを与えて正しいチャンクタグを決定するものである。

In [17]:
class UnigramChunker(nltk.ChunkParserI):
 def __init__(self, train_sents):
   train_data = [[(t,c) for w,t,c in nltk.chunk.tree2conlltags(sent)] 
                 for sent in train_sents]
   self.tagger = nltk.UnigramTagger(train_data)

 def parse(self, sentence):
   pos_tags = [pos for (word,pos) in sentence]
   tagged_pos_tags = self.tagger.tag(pos_tags)
   chunktags = [chunktag for (pos, chunktag) in tagged_pos_tags]
   conlltags = [(word, pos, chunktag) for ((word,pos),chunktag)
                in zip(sentence, chunktags)] 
   return nltk.chunk.conlltags2tree(conlltags)

unigram_chunker = UnigramChunker(train_sentences)
print(unigram_chunker.evaluate(test_sentences))
print(unigram_chunker.tagger.tag(postags))

<ipython-input-17-095b0c03af83>:16: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  print(unigram_chunker.evaluate(test_sentences))


ChunkParse score:
    IOB Accuracy:  92.9%%
    Precision:     79.9%%
    Recall:        86.8%%
    F-Measure:     83.2%%
[('#', 'B-NP'), ('$', 'B-NP'), ("''", 'O'), ('(', 'O'), (')', 'O'), (',', 'O'), ('.', 'O'), (':', 'O'), ('CC', 'O'), ('CD', 'I-NP'), ('DT', 'B-NP'), ('EX', 'B-NP'), ('FW', 'I-NP'), ('IN', 'O'), ('JJ', 'I-NP'), ('JJR', 'B-NP'), ('JJS', 'I-NP'), ('MD', 'O'), ('NN', 'I-NP'), ('NNP', 'I-NP'), ('NNPS', 'I-NP'), ('NNS', 'I-NP'), ('PDT', 'I-NP'), ('POS', 'B-NP'), ('PRP', 'B-NP'), ('PRP$', 'B-NP'), ('RB', 'O'), ('RBR', 'O'), ('RBS', 'B-NP'), ('RP', 'O'), ('TO', 'O'), ('UH', 'B-NP'), ('VB', 'O'), ('VBD', 'O'), ('VBG', 'O'), ('VBN', 'O'), ('VBP', 'O'), ('VBZ', 'O'), ('WDT', 'B-NP'), ('WP', 'B-NP'), ('WP$', 'B-NP'), ('WRB', 'O'), ('``', 'O')]


正規表現チャンカではF値が69.2%だったが、今回は83.2%まで上昇した。再現率が特に高くなっている。訓練コーパスから単語の品詞タグとチャンクタグの関連性を学習することで、与えられた品詞タグから、その品詞に対して適切なチャンクタグを予測できた。また、出力されたタグのペアを見ると、句読点はチャンクの外にあり、NPチャンクに含まれる品詞はほとんどが名詞であり、限定詞や所有格からNPチャンクが始まることを確認できる。ただし、単語単体での情報に依存しているため、文脈や依存関係などのより高度な言語特徴を考慮することはできない。そこで、バイグラムタガーを用いる。

### 3-7. バイグラムタガーを用いた名詞句チャンカ


In [18]:
class BigramChunker(nltk.ChunkParserI):
 def __init__(self, train_sents):
   train_data = [[(t,c) for w,t,c in nltk.chunk.tree2conlltags(sent)] 
                 for sent in train_sents]
   self.tagger = nltk.BigramTagger(train_data)

 def parse(self, sentence):
   pos_tags = [pos for (word,pos) in sentence]
   tagged_pos_tags = self.tagger.tag(pos_tags)
   chunktags = [chunktag for (pos, chunktag) in tagged_pos_tags]
   conlltags = [(word, pos, chunktag) for ((word,pos),chunktag)
                in zip(sentence, chunktags)] 
   return nltk.chunk.conlltags2tree(conlltags)
   
bigram_chunker = BigramChunker(train_sentences)
print(bigram_chunker.evaluate(test_sentences))

<ipython-input-18-e55b2745ee29>:16: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  print(bigram_chunker.evaluate(test_sentences))


ChunkParse score:
    IOB Accuracy:  93.4%%
    Precision:     82.6%%
    Recall:        87.5%%
    F-Measure:     85.0%%


直線の単語を踏まえて文脈を考慮できるようになったことで、4つの評価値全てが上昇した。

今までのチャンカでは、品詞タグをもとにチャンクを生成していた。しかし、これではSVOO構文などを正確にチャンキングできない可能性がある。そこで、品詞タグを使うだけではなく、単語の内容に関する情報も使うことにする。

### 3-8. 連続分類器を利用したチャンカ


In [19]:
"""
class ConsecutiveNPChunkTagger(nltk.TaggerI):

    def __init__(self, train_sents):
        train_set = []
        for tagged_sent in train_sents:
            untagged_sent = nltk.tag.untag(tagged_sent)
            history = []
            for i, (word, tag) in enumerate(tagged_sent):
                featureset = npchunk_features(untagged_sent, i, history)
                train_set.append( (featureset, tag) )
                history.append(tag)
        self.classifier = nltk.MaxentClassifier.train(
            train_set, algorithm='megam', trace=0)

    def tag(self, sentence):
        history = []
        for i, word in enumerate(sentence):
            featureset = npchunk_features(sentence, i, history)
            tag = self.classifier.classify(featureset)
            history.append(tag)
        return zip(sentence, history)

class ConsecutiveNPChunker(nltk.ChunkParserI):
    def __init__(self, train_sents):
        tagged_sents = [[((w,t),c) for (w,t,c) in
                         nltk.chunk.tree2conlltags(sent)]
                        for sent in train_sents]
        self.tagger = ConsecutiveNPChunkTagger(tagged_sents)

    def parse(self, sentence):
        tagged_sents = self.tagger.tag(sentence)
        conlltags = [(w,t,c) for ((w,t),c) in tagged_sents]
        return nltk.chunk.conlltags2tree(conlltags)

def npchunk_features(sentence, i, history):
    word, pos = sentence[i]
    return {"pos": pos}

chunker = ConsecutiveNPChunker(train_sentences)
"""

'\nclass ConsecutiveNPChunkTagger(nltk.TaggerI):\n\n    def __init__(self, train_sents):\n        train_set = []\n        for tagged_sent in train_sents:\n            untagged_sent = nltk.tag.untag(tagged_sent)\n            history = []\n            for i, (word, tag) in enumerate(tagged_sent):\n                featureset = npchunk_features(untagged_sent, i, history)\n                train_set.append( (featureset, tag) )\n                history.append(tag)\n        self.classifier = nltk.MaxentClassifier.train(\n            train_set, algorithm=\'megam\', trace=0)\n\n    def tag(self, sentence):\n        history = []\n        for i, word in enumerate(sentence):\n            featureset = npchunk_features(sentence, i, history)\n            tag = self.classifier.classify(featureset)\n            history.append(tag)\n        return zip(sentence, history)\n\nclass ConsecutiveNPChunker(nltk.ChunkParserI):\n    def __init__(self, train_sents):\n        tagged_sents = [[((w,t),c) for (w,t,c) 

参考書に沿って以上を実装してみたが
```
LookupError: 
NLTK was unable to find the megam file!
Use software specific configuration parameters or set the MEGAM environment variable.
```
のエラーが出てしまった。おそらくPythonとNLTKのバージョンの違いが原因だと考える。使用しているバージョンに合わせた実装を調べるなどして試行錯誤したが、解決できなかった。



これまでは様々な名詞句チャンカを見てきた。次は、名詞句・前置詞句・動詞句、そして文を扱うルールを構築する。

### 3-9. NP, PP, VP, Sを扱うチャンカ
新たにルールを追加し、実装は3-5と同じようにした。

In [20]:
sentence = [("Mary", "NN"), ("saw", "VBD"), ("the", "DT"), ("cat", "NN"), ("sit", "VB"), ("on", "IN"), ("the", "DT"), ("mat", "NN")]
grammar = r"""
    NP: {<DT|JJ|NN.*>+}
    PP: {<IN><NP>}
    VP: {<VB.*><NP|PP|CLAUSE>+$}
    CLAUSE: {<NP><VP>}
    """
cp = nltk.RegexpParser(grammar)
print(cp.parse(sentence))

(S
  (NP Mary/NN)
  saw/VBD
  (CLAUSE
    (NP the/DT cat/NN)
    (VP sit/VB (PP on/IN (NP the/DT mat/NN)))))


![チャンキングの構造木](https://drive.google.com/uc?id=15M5Z2HNrg7Al5mI8paIwCIIUQywqdPoK)  

ここでは、sawから始まる動詞句を見逃してしまっている。そこで、すべてのパターンを試した後、チャンカがその処理を繰り返すようにする。

In [21]:
cp = nltk.RegexpParser(grammar, loop=2)
print(cp.parse(sentence))

(S
  (CLAUSE
    (NP Mary/NN)
    (VP
      saw/VBD
      (CLAUSE
        (NP the/DT cat/NN)
        (VP sit/VB (PP on/IN (NP the/DT mat/NN)))))))


![チャンキングの構造木](https://drive.google.com/uc?id=1J4ag1o3s1czZCGNIKy7WNJD78Q-783Pa)  


1段階ではなく再帰的にルールを適用することで、見落としていた句を拾えるようになった。

# 4. 構文解析
最後に、構文解析を扱う。ここでは、昨日の試験に登場した「John gave Susan a book in the room」という文について、試験と同じ文法ルールを用いて構文解析を行う。

### 4-1. 異なる構文解析アルゴリズムの比較

In [22]:
from nltk import CFG

grammar = CFG.fromstring("""
    S -> NP VP | S PP
    NP -> noun | det noun | NP PP
    VP -> verb NP | verb NP NP | VP PP
    PP -> prep NP
    noun -> "John" | "Susan" | "book" | "room"
    verb -> "gave"
    det -> "a" | "the"
    prep -> "in"
""")
sentence = 'John gave Susan a book in the room'.split()

# parser = nltk.RecursiveDescentParser(grammar) # 再帰下降構文解析
# parser = nltk.ShiftReduceParser(grammar) # Shift-Reduce構文解析
parser = nltk.ChartParser(grammar) # チャート法構文解析

for tree in parser.parse(sentence):
  print(tree)

(S
  (S
    (NP (noun John))
    (VP (verb gave) (NP (noun Susan)) (NP (det a) (noun book))))
  (PP (prep in) (NP (det the) (noun room))))
(S
  (NP (noun John))
  (VP
    (VP (verb gave) (NP (noun Susan)) (NP (det a) (noun book)))
    (PP (prep in) (NP (det the) (noun room)))))
(S
  (NP (noun John))
  (VP
    (verb gave)
    (NP (noun Susan))
    (NP
      (NP (det a) (noun book))
      (PP (prep in) (NP (det the) (noun room))))))


RecursiveDescentParser（再帰下降構文解析器）の場合、RecursionErrorとなってしまって、適切に構文解析ができなかった。再帰下降構文解析は、トップダウン方式で、文法の各規則に対して再帰的に呼び出される関数を用いて文を解析する。NP -> NP PPのような左再帰的なルールを用いると、バックトラックを繰り返し無限ループに陥ってしまう。今回のルールでは、NP -> NP PPとVP -> VP PPという2つの左再帰ルールがあり、再帰の無限ループによるエラーが起きてしまった。

ShiftReduceParser（Shift-Reduce構文解析器）の場合、結果が何も表示されなかった。Shift-Reduce構文解析は、ボトムアップ方式で
、スタックとトークン列の操作を用いて文を解析する。トークン列を先頭から順にスキャンし、スタック上でルールにマッチするかどうかをチェックする。ルールにマッチする場合、スタック上のトークンをルールの左辺に置き換え、ルールにマッチしない場合、スキャンしたトークンをスタックにプッシュする。このシフト（スキャン）、リデュース（置き換え）の操作を繰り返しながら解析を進める。ただし、行き止まりに達して構文木を見つけられなくなってしまう場合がある。なぜなら、構文解析器がより初期の段階でなされた選択を取り消すことができないからだ。今回のルールは結果が3つ表示される通り、曖昧な文法が含まれる。複数のルールが同時に適用可能な場合にどちらかを選択する必要があり、その結果行き止まりに達してしまったと考える。

ChartParser（チャート構文解析器）の場合、望み通りの結果が得られた。チャート構文解析器は、ボトムアップの動的計画法に基づいた手法である。入力文をスキャンし、可能なすべての部分構文木を生成し、チャートと呼ばれるデータ構造に格納する。チャート内の部分構文木を組み合わせて、最終的な解析結果を得る。全体的な解析の効率性が高く、再帰深さの制限に対して頑健であることから、適切に解析できた。

### 4-2. 構文解析に確率を与える
試験問題と同様、それぞれのルールに確率を与えた。確率値は基本的に試験問題と同じであるが、各品詞におけるある単語の確率は、その品詞全体で合計1にする必要があったため、適当に値を与えた。

In [23]:
from nltk.grammar import PCFG
from nltk.parse import pchart

grammar = PCFG.fromstring("""
    S -> NP VP [0.6] | S PP [0.4]
    NP -> noun [0.3] | det noun [0.5] | NP PP [0.2]
    VP -> verb NP [0.5] | verb NP NP [0.4] | VP PP [0.1]
    PP -> prep NP [1.0]
    noun -> "John" [0.1] | "Susan" [0.1] | "book" [0.4] | "room" [0.4]
    verb -> "gave" [1.0]
    det -> "a" [0.5] | "the" [0.5]
    prep -> "in" [1.0]
""")
parser = pchart.InsideChartParser(grammar)
for tree in parser.parse(sentence):
  print(tree)

(S
  (S
    (NP (noun John))
    (VP
      (verb gave)
      (NP (noun Susan))
      (NP (det a) (noun book))))
  (PP (prep in) (NP (det the) (noun room)))) (p=8.64e-07)
(S
  (NP (noun John))
  (VP
    (verb gave)
    (NP (noun Susan))
    (NP
      (NP (det a) (noun book))
      (PP (prep in) (NP (det the) (noun room)))))) (p=4.32e-07)
(S
  (NP (noun John))
  (VP
    (VP
      (verb gave)
      (NP (noun Susan))
      (NP (det a) (noun book)))
    (PP (prep in) (NP (det the) (noun room))))) (p=2.16e-07)


結果として、「ジョンはこの部屋でスーザンに本をあげた」と訳されるような構文木の確率が最も高くなった。

### 4-3. 適格部分文字列表の作成
4-*1ではチャート構文解析器を用いて解析を行った。この解析器で用いる適格部分文字列表を、構文解析関数を使わずに実装した。*

In [24]:
def init_wfst(tokens, grammar):
     numtokens = len(tokens)
     wfst = [[None for i in range(numtokens+1)] for j in range(numtokens+1)]
     for i in range(numtokens):
         productions = grammar.productions(rhs=tokens[i])
         wfst[i][i+1] = productions[0].lhs()
     return wfst

def complete_wfst(wfst, tokens, grammar, trace=False):
     index = dict((p.rhs(), p.lhs()) for p in grammar.productions())
     numtokens = len(tokens)
     for span in range(2, numtokens+1):
         for start in range(numtokens+1-span):
             end = start + span
             for mid in range(start+1, end):
                 nt1, nt2 = wfst[start][mid], wfst[mid][end]
                 if nt1 and nt2 and (nt1,nt2) in index:
                     wfst[start][end] = index[(nt1,nt2)]
                     if trace:
                         print("[{}] {} [{}] {} [{}] ==> [{}] {} [{}]".format(start, nt1, mid, nt2, end, start, index[(nt1,nt2)], end))
     return wfst

def display(wfst, tokens):
    num_tokens = len(wfst) - 1

    headers = ' '.join([("%-4d" % i) for i in range(1, num_tokens + 1)])
    print('WFST ' + headers)

    for i in range(num_tokens):
        row = "{:<4d}".format(i)
        for j in range(1, num_tokens + 1):
            cell = "{}".format(wfst[i][j] or '.')
            row += "{:<5s}".format(cell)
        print(row)

grammar = CFG.fromstring("""
    S -> NP VP | S PP
    NP -> noun | det noun | NP PP
    VP -> verb NP | verb NP NP | VP PP
    PP -> prep NP
    noun -> "John" | "Susan" | "book" | "room"
    verb -> "gave"
    det -> "a" | "the"
    prep -> "in"
""")

wfst0 = init_wfst(sentence, grammar)

wfst1 = complete_wfst(wfst0, sentence, grammar)
display(wfst1, sentence)

wfst1 = complete_wfst(wfst0, sentence, grammar, trace=True)

WFST 1    2    3    4    5    6    7    8   
0   noun .    .    .    .    .    .    .    
1   .    verb .    .    .    .    .    .    
2   .    .    noun .    .    .    .    .    
3   .    .    .    det  NP   .    .    NP   
4   .    .    .    .    noun .    .    .    
5   .    .    .    .    .    prep .    PP   
6   .    .    .    .    .    .    det  NP   
7   .    .    .    .    .    .    .    noun 
[3] det [4] noun [5] ==> [3] NP [5]
[6] det [7] noun [8] ==> [6] NP [8]
[5] prep [6] NP [8] ==> [5] PP [8]
[3] NP [5] PP [8] ==> [3] NP [8]


[0] John [1] gave [2] Susan [3] a [4] book [5] in [6] the [7] room [8]  
適格部分文字列表では、三角行列中のセルを埋めていくことによって単語の位置を記録する。縦軸は部分文字列の開始位置、横軸は終了位置を示す。結果として、例えば「gave」を表すverbは座標(1, 2)のセルに現れている。また、「a」を表すdetは座標(3, 4)に、「book」を表すnounは座標(4, 5)に現れている。そして、「a book」を表すNPは座標(3, 5)に現れている。一般化すると、生成規則A -> B Cがある時、非終端記号Bが(i, k)に、Cが(k, j)にある場合、Aは(i, j)に位置する。ただし、今回の実装方法では複数の解釈が得られる曖昧な文法を完全に再現することはできない。そのため、部分木は表現できているが、文章全体を表すSは表に出てこない。